In [32]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import isnan, when, count, col
from pyspark.mllib.stat import Statistics
from pyspark.mllib.feature import StandardScaler, PCA
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StringType,BooleanType,DateType

In [2]:
spark = SparkSession.builder.master('local[1]').appName('Teste.com').getOrCreate()

In [3]:
spark

In [38]:
df = spark.read.csv("/Users/aurelianosancho/Dropbox/Git/Exemplos/train.csv",header=True)

In [27]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [19]:
df.show(20)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [15]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [39]:
l = ['PassengerId','Survived','Pclass','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

def formato(dt, col_names):
    for i in col_names:
        dt = dt.withColumn(i,col(i).cast("float"))
    return dt

In [40]:
df = formato(df,l)

root
 |-- PassengerId: float (nullable = true)
 |-- Survived: float (nullable = true)
 |-- Pclass: float (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: float (nullable = true)
 |-- Parch: float (nullable = true)
 |-- Ticket: float (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: float (nullable = true)
 |-- Embarked: float (nullable = true)



In [42]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|   230|   0|  891|     891|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [38]:
df.filter(df.artists.startswith("a")).show()

+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+
|valence|year|acousticness|artists|danceability|duration_ms|energy|explicit| id|instrumentalness|key|liveness|loudness|mode|name|popularity|release_date|speechiness|tempo|Nova|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+------------+-----------+-----+----+



In [40]:
df.filter(df.artists.contains("N")).show(2)

+-------+----+------------+--------------------+------------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+------------+----------+------------+-----------+-------+----+
|valence|year|acousticness|             artists|      danceability|duration_ms|             energy|explicit|                  id|instrumentalness|key|liveness|           loudness|mode|        name|popularity|release_date|speechiness|  tempo|Nova|
+-------+----+------------+--------------------+------------------+-----------+-------------------+--------+--------------------+----------------+---+--------+-------------------+----+------------+----------+------------+-----------+-------+----+
| 0.0394|1921|       0.961|['KHP Kridhamarda...|             0.328|     500062|              0.166|       0|1o6I8BglA6ylDMrIE...|           0.913|  3|   0.101|             -14.85|   1|   Gati Bali|         5|        1921|     0.0339|110.339|-5.0|
|  0.196|192

In [27]:
df.createOrReplaceTempView("Tabela1")

In [30]:
df2 = spark.sql("SELECT artists,popularity from Tabela1")

In [31]:
df2.show()

+--------------------+----------+
|             artists|popularity|
+--------------------+----------+
|['Sergei Rachmani...|         4|
|      ['Dennis Day']|         5|
|['KHP Kridhamarda...|         5|
|    ['Frank Parker']|         3|
|      ['Phil Regan']|         2|
|['KHP Kridhamarda...|         6|
|  ['John McCormack']|         4|
|['Sergei Rachmani...|         2|
| ['Ignacio Corsini']|         0|
|         ['Fortugé']|         0|
|['Maurice Chevali...|         0|
| ['Ignacio Corsini']|         0|
|         ['Georgel']|         0|
| ['Mehmet Kemiksiz']|         0|
|      ['Zay Gatsby']|         0|
|['Sergei Rachmani...|         4|
|      ['Phil Regan']|         1|
|['Hector Berlioz'...|         0|
|         ['THE GUY']|         0|
|['Christopher Lyn...|         0|
+--------------------+----------+
only showing top 20 rows

